In [1]:
import torch.nn as nn
import torch
import MST
import numpy as np

In [2]:
np.random.seed(42)

In [3]:
in_C = 1
out_C = 16
groups = 1
stride = 1
padding = 50
dilation = 1
kernel = 10

params = [in_C, out_C, kernel, stride, padding, dilation, groups]

print(f"{(16 - kernel + padding*2 - (kernel-1)*(dilation-1))/stride + 1}")

107.0


In [4]:
with torch.no_grad():
    myConv = MST.Conv2d(*params)
    torchConv = nn.Conv2d(*params)

    torchConv.weight = nn.Parameter(torch.tensor(myConv._w))
    torchConv.bias = nn.Parameter(torch.tensor(myConv._bias).flatten())


(100, 100)


In [5]:
image = np.ones((1, in_C, 16, 16))

In [6]:
with torch.no_grad():
    myres = myConv(image)
    nnres = torchConv(torch.Tensor(image))
    print(myres.shape)
    print(nnres.shape)

    nnres = nnres.numpy().round(3).flatten()
    myres = myres.round(3).flatten()
    bads = np.where(abs(nnres - myres) > 0.001)
    print(myres[:20])
    print(nnres[:20])
    for pos in bads[0][:10]:
        print(f"[{pos}]: {myres[pos]:.3f} {nnres[pos]:.3f}")

(1, 16, 108, 108)
torch.Size([1, 16, 107, 107])


ValueError: operands could not be broadcast together with shapes (183184,) (186624,) 

In [ ]:
%%timeit
myres = myConv(image)

5.48 ms ± 173 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
with torch.no_grad():
    nnres = torchConv(torch.Tensor(image))

100 µs ± 2.72 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
